In [6]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

In [12]:
rainfall_path = 'rainfall.xlsx'  # Replace with your CSV file path
num_features = 1  # Assuming each time step has one feature (you may need to adjust this)
embedding_size = 64  # Adjust the embedding size as needed

# Load CSV data
data = pd.read_excel(rainfall_path)
print(data)
# Assuming the CSV file has columns for each time series and rows for each time step
# You may need to adjust this depending on the structure of your CSV file
num_time_series = len(data.columns)
num_time_steps = len(data)

# Convert data to PyTorch tensor
data_tensor = torch.tensor(data.values, dtype=torch.float32)

# Reshape the data tensor to match the expected input dimensions of the EmbeddingModule
data_tensor = data_tensor.view(1, num_time_steps, 1, num_time_series)

     Unnamed: 0  Bãi Cháy  Cà Mau  Đà Lạt  Đà Nẵng  Hà Nội     Huế  Lai Châu  \
0             0       5.0    4.00   10.74     33.0     9.0    73.0      73.0   
1             1      21.0   28.41   14.00     64.0    24.0    31.0      53.0   
2             2      12.0    1.00  120.00      4.0    11.0    22.0      78.0   
3             3      17.0    4.00  102.00    113.0    59.0    56.0     114.0   
4             4     413.0  274.00  123.00     39.0   214.0   242.0     341.0   
..          ...       ...     ...     ...      ...     ...     ...       ...   
247         247     708.4  228.30  280.60    187.0   486.3   157.5     257.9   
248         248     461.2  409.20  293.70    419.4   242.0   448.8     166.1   
249         249     210.2  352.70  159.50   1219.6    84.4  1366.5      72.9   
250         250      34.9  313.30   99.10    156.6     7.8   226.4     111.0   
251         251       1.5   71.90   37.90    349.7    13.7   786.6      72.6   

     Nam Định  Nha Trang  Pleiku  Quy N

In [16]:
class EmbeddingModule(nn.Module):
    def __init__(self, num_features, embedding_size):
        super().__init__()
        self.linear = nn.Linear(num_features, embedding_size)

    def forward(self, x):
        # Reshape input into dimensions (batch_size, num_features, num_time_series, num_time_steps)
        x = x.permute(0, 3, 1, 2)   

        # Apply feature-wise linear embedding
        x = self.linear(x)
        
        return x   
embedding_module = EmbeddingModule(num_features, embedding_size)

# Apply the embedding module to the data
embedding_result = embedding_module(data_tensor)
print("Embedded data shape:", embedding_result.shape)

Embedded data shape: torch.Size([1, 16, 252, 64])


In [5]:
class EmbeddingModule(nn.Module):
    def __init__(self, num_features, embedding_size):
        super().__init__()
        self.linear = nn.Linear(num_features, embedding_size)

    def forward(self, x):
        # Reshape input into dimensions (batch_size, num_features, num_time_series, num_time_steps)
        x = x.permute(0, 3, 1, 2)   

        # Apply feature-wise linear embedding
        x = self.linear(x)
        
        return x   

class LTCN(nn.Module):
    def __init__(self, embedding_size, num_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(embedding_size, num_channels, kernel_size=(1,5))
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=(1, 3))

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        
        # Keep only last timestep 
        x = x[..., -1:]  
        
        return x

class GLFormer(nn.Module):
    def __init__(self, embedding_size): 
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(embedding_size, nhead=4)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
    def forward(self, x):
        x = x.permute(1, 0, 2, 3)    
        x = self.transformer(x) 
        return x.permute(1, 0, 2, 3)
    
class LightCTS(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = EmbeddingModule(num_features=2, embedding_size=64)   
        self.ltcn = LTCN(embedding_size=64, num_channels=32)
        self.glformer = GLFormer(embedding_size=64)
        
        self.linear = nn.Linear(64, 1) 

    def forward(self, x):
        x = self.embedding(x) 
        x = self.ltcn(x) 
        x = self.glformer(x)
        
        # Flatten across time series dimension
        x = torch.flatten(x, start_dim=1)  
        return self.linear(x)   

model = LightCTS() 